 # Проект "Анализ паттернов". Построение алгоритмов кластеризации.

In [1]:
# imports
import pyexcel_xlsx as xl
import pandas as pd
import numpy as np
from itertools import permutations
from itertools import combinations

## Проект алгоритма кластеризации №1.

__Основа метода - расстояние Хемминга.__

Описание:

1) подгрузка данных из excel 
    
2) создание дата фрейма регион-партия
    - проверяем файл на ошибки (пропуски, сумма ряда больше 100%, данные другого типа)
3) кодировка регионов (0-1-2)

4) подсчет попарного расстояния Хемминга для каждой пары регионов без перестановок
    - регионы, имеющие нулевое расстояние между собой, относятся к одному паттерну
    
5) визуализация полученного результата
    - график
    - текст

In [4]:
file_path = '/Users/macbook/Desktop/Par_reg.xlsx' # загрузка данных 
general_df = xl.get_data(file_path)

for country_year in general_df.keys():
    
    df = general_df[country_year]
    print(country_year)
    print('--------------------------------------------------------------')

    df[0] = df[0][1:] # создание матрицы регион-партия
    parties_names = df[0][0:df[0].index('')]
    regions_names = []
    data = []

    for element in df[1:]:
        regions_names.append(element[0])
        data.append(element[1:])

    pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data)

    # проверка на ошибки

    counter = 0
    print('ERRORS')
    for dtype in pd_df.dtypes:
        if dtype != 'int64':
            print('Party {} contains a non-numeric value'.format(parties_names[counter]))
        counter += 1
        
    print('--------------------------------------------------------------')
    print()

    pd_df = pd_df.convert_objects(convert_numeric = True) # замена всех нечисловых значений на 0.0
    pd_df.replace(to_replace = np.nan, value = 0, inplace = True)

    counter = 0
    for votes_sum in pd_df.sum(axis = 1):
        if abs(100 - votes_sum) > 1.0:
            print('Sum of region {} votes does not equal 100%, but {}'.format(regions_names[counter], votes_sum))
        counter += 1

    print('--------------------------------------------------------------')
    print()    
        
    codes = [] # список с кодировками всех регионов (список списков)   
    for i in range(pd_df.shape[0]): # для всех регионов в матрице
        code = ''
        for j in range(pd_df.shape[1] - 1): # для всех партий в регионе
            if pd_df.iloc[i, j] > pd_df.iloc[i, j + 1]:
                code += '2'
            elif pd_df.iloc[i, j] == pd_df.iloc[i, j + 1]:
                code += '0'
            else:
                code += '1'
        codes.append(code)

    pd_df['code'] = codes

    unique_codes = np.unique(codes) # список уникальных кодировок
    print('Можно выделить {} кластера(-ов)'.format(len(unique_codes)))
    print()
    print('--------------------------------------------------------------')
    print()

    counter = 1
    for unique_code in unique_codes:
        print('К кластеру {}, выраженному кодировкой {}, принадлежат регионы:'.format(counter, unique_code))
        for region in pd_df.loc[pd_df['code'] == unique_code].index:
            print(region)
        counter += 1
        print()
        print('--------------------------------------------------------------')
        print()
    print('==============================================================')
    print()

Лист1
--------------------------------------------------------------
ERRORS
--------------------------------------------------------------

Sum of region Region 2 votes does not equal 100%, but 90
Sum of region Region 3 votes does not equal 100%, but 65
--------------------------------------------------------------

Можно выделить 3 кластера(-ов)

--------------------------------------------------------------

К кластеру 1, выраженному кодировкой 00, принадлежат регионы:
Region 2

--------------------------------------------------------------

К кластеру 2, выраженному кодировкой 11, принадлежат регионы:
Region 3
Region 4
Region 5
Region 6

--------------------------------------------------------------

К кластеру 3, выраженному кодировкой 21, принадлежат регионы:
Region 1

--------------------------------------------------------------




/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


### Проект алгоритма кластеризации №2.

Инвариантный способ

Описание:

1) подгрузка данных из excel 
       - проверяем файл на ошибки (пропуски, сумма ряда больше 100%, данные другого типа) --> raise error
2) создаем data frame
3) функция кодировки регионов (0-1-2)
4) для каждой возможной перестановки столбцов фрейма
       - кодировка
       - расстояния Хемминга
       - развиение на паттерны
5) если во всех перестановках гергион относится к одному паттерну, выделяется инвариативный паттерн
6) визуализация полученного результата
        - график
        - текст

In [7]:
file_path = '/Users/macbook/Desktop/Par_reg.xlsx' # загрузка данных 
general_df = xl.get_data(file_path)

for country_year in general_df.keys(): # проходим по каждому листу в excel файле
    
    df = general_df[country_year]
    print('--------------------------------------------------------------')
    print(country_year)
    print('--------------------------------------------------------------')
    print()

    df[0] = df[0][1:] # создание матрицы регион-партия
    parties_names = df[0][0:df[0].index('')]
    regions_names = []
    data = []

    for element in df[1:]:
        regions_names.append(element[0])
        data.append(element[1:])

    pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data)
    
    all_clusters = []

    # проверка на ошибки

    counter = 0
    for dtype in pd_df.dtypes:
        if dtype != 'int64':
            print('Party {} contains a non-numeric value'.format(parties_names[counter]))
        counter += 1
        
    print('--------------------------------------------------------------')
    print()

    pd_df = pd_df.convert_objects(convert_numeric = True) # замена всех нечисловых значений на 0.0
    pd_df.replace(to_replace = np.nan, value = 0, inplace = True)

    counter = 0
    for votes_sum in pd_df.sum(axis = 1):
        if abs(100 - votes_sum) > 1.0:
            print('Sum of region {} votes does not equal 100%, but {}'.format(regions_names[counter], votes_sum))
        counter += 1

    print('--------------------------------------------------------------')
    print()   
    
    parties_dict = {} # создаем словарь партия - голоса во всех регионах
    for party in pd_df.columns:
        parties_dict[party] = np.array(pd_df[party])
    
    for permutation in permutations(parties_names): # для всех возможных перестановок партий создаем новый датафрейм
        pd_df_perm = pd.DataFrame(index = regions_names, columns = permutation)
        for column in pd_df_perm.columns:
            pd_df_perm[column] = parties_dict[column]
            
        clusters_perm = []
            
        codes = [] # список с кодировками всех регионов (список списков)   
        for i in range(pd_df_perm.shape[0]): # для всех регионов в матрице
            code = ''
            for j in range(pd_df_perm.shape[1] - 1): # для всех партий в регионе
                if pd_df_perm.iloc[i, j] > pd_df_perm.iloc[i, j + 1]:
                    code += '2'
                elif pd_df_perm.iloc[i, j] == pd_df_perm.iloc[i, j + 1]:
                    code += '0'
                else:
                    code += '1'
            codes.append(code)

        pd_df_perm['code'] = codes

        unique_codes = np.unique(codes) # список уникальных кодировок

        counter = 1
        for unique_code in unique_codes:
            cluster = []
            for region in pd_df_perm.loc[pd_df_perm.code == unique_code].index:
                cluster.append(region)
            counter += 1
            clusters_perm.append(cluster)
        all_clusters.append(clusters_perm)
        
    all_possible_permutations = []
    for n in range(2, len(regions_names) + 1):
        for i in combinations(regions_names, n):
            all_possible_permutations.append(list(i))
            
    invariant_clusters = []
    for possible_permutation in all_possible_permutations:
        counter_1 = 0
        for permut_clusters in all_clusters:
            counter_2 = 0
            for cluster in permut_clusters:
                if set(possible_permutation).issubset(set(cluster)):
                    counter_2 += 1
            if counter_2 > 0:
                counter_1 += 1
        if counter_1 == len(all_clusters):
            invariant_clusters.append(possible_permutation)
            
    max_len = 0
    longest_list_index = 0
    for index in range(len(invariant_clusters)):
        list_len = len(invariant_clusters[index])
        if list_len >= max_len:
            max_len = list_len
            longest_list_index = index

    new_invariant_clusters = []
    longest_cluster = invariant_clusters[longest_list_index]
    invariant_clusters.remove(invariant_clusters[longest_list_index])
    for cluster in invariant_clusters:
        if set(cluster).issubset(set(longest_cluster)) != True:
            new_invariant_clusters.append(cluster)
    new_invariant_clusters.append(longest_cluster)
    
    print('{} invariant cluster(s) detected'.format(len(new_invariant_clusters)))
    print()
    for inv_cluster in new_invariant_clusters:
        print(inv_cluster)
        
    print('==============================================================')
    print()

--------------------------------------------------------------
Лист1
--------------------------------------------------------------

--------------------------------------------------------------

Sum of region Region 2 votes does not equal 100%, but 90
Sum of region Region 3 votes does not equal 100%, but 65
--------------------------------------------------------------

1 invariant cluster(s) detected

['Region 3', 'Region 4', 'Region 5', 'Region 6']



/Users/macbook/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
